In [18]:
%pip install matplotlib tpot

from scipy.stats import norm
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()

import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.


# 10회차 과제
목표: KNN을 이용한 모델을 만들되, grid search를 사용하여 hyperparameter를 결정하고 K-fold cross validation(K=10)으로 테스트하기.  
(수업 자료의 `과제 연계 실습` 참고)

## 와인 분류하기

다음은 와인과 관련한 여러 정보와 음용 가능 여부가 담긴 데이터이다.  
KNN과 grid search를 사용하여 종류('target')를 예측하되, 최소 95%의 정확도(K-fold cross validation, K=10 기준)를 달성하시오.

In [24]:
raw_wine = datasets.load_wine()
wine = pd.DataFrame(raw_wine['data'], index=raw_wine['target'], columns=raw_wine['feature_names'])
wine = wine.reset_index(names=['target'] + raw_wine['feature_names'])
display(wine)

#전 과제에서 갖고옴 
new_columns_names = ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


X = wine[new_columns_names]
y = wine['target']



from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 10, shuffle = False)

from sklearn.pipeline import Pipeline
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", classifier),
])

grid_vals = {'model__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
grid_knn = GridSearchCV(estimator=pipe, param_grid=grid_vals, scoring='accuracy', 
                       cv=kfold, refit=True, return_train_score=True) 


grid_knn.fit(X, y)
display(grid_knn.cv_results_)

kfold = StratifiedKFold(n_splits = 10, shuffle = False)

accs = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y[train_idx], y[test_idx]

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)

    scaler = StandardScaler()
    scaler.fit(X_test)
    X_test_scaled = scaler.transform(X_test)

    random_knn.best_estimator_.fit(X_train_scaled, y_train)
    y_pred = random_knn.best_estimator_.predict(X_test_scaled)

    print(confusion_matrix(y_test, y_pred))
    print(f"[Fold no.{fold}]: {accuracy_score(y_test, y_pred)*100:.3f}%")

    accs.append(accuracy_score(y_test, y_pred))

print(f"평균 K-fold cross validation (best model): {sum(accs)/len(accs)*100:.3f}%")



,target,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,0,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,0,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,0,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,0,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,2,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,2,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,2,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


{'mean_fit_time': array([0.00130048, 0.00070255, 0.000611  , 0.00060475, 0.0006089 ,
        0.00061443, 0.00062733, 0.00059519, 0.00059514, 0.00059595]),
 'std_fit_time': array([5.48995546e-04, 1.24909860e-04, 1.30321654e-05, 1.27482617e-05,
        1.21113118e-05, 1.52864987e-05, 8.75347441e-05, 1.04916095e-05,
        7.51665382e-06, 8.91647236e-06]),
 'mean_score_time': array([0.00131872, 0.00074494, 0.00066936, 0.00067306, 0.00067677,
        0.00068617, 0.00066411, 0.00066686, 0.0006583 , 0.0006789 ]),
 'std_score_time': array([3.28906591e-04, 1.60171094e-04, 1.22544598e-05, 1.14097677e-05,
        2.00331770e-05, 3.45481602e-05, 2.21673512e-05, 1.18035257e-05,
        1.07115766e-05, 2.55417970e-05]),
 'param_model__n_neighbors': masked_array(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'params': [{'model__n_neighbors': 1},
  {'model__n_neighbors':

[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.0]: 100.000%
[[6 0 0]
 [1 6 0]
 [0 0 5]]
[Fold no.1]: 94.444%
[[6 0 0]
 [1 6 0]
 [0 0 5]]
[Fold no.2]: 94.444%
[[6 0 0]
 [0 6 1]
 [0 0 5]]
[Fold no.3]: 94.444%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.4]: 100.000%
[[6 0 0]
 [0 6 1]
 [0 0 5]]
[Fold no.5]: 94.444%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.6]: 100.000%
[[6 0 0]
 [0 7 0]
 [0 0 5]]
[Fold no.7]: 100.000%
[[6 0 0]
 [1 5 1]
 [0 0 4]]
[Fold no.8]: 88.235%
[[5 0 0]
 [0 8 0]
 [0 0 4]]
[Fold no.9]: 100.000%
평균 K-fold cross validation (best model): 96.601%
